In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import random
from collections import Counter
import doctorailib
from doctorailib import pre_processing, doctorai

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/user/venvs/drAI+/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Prepare MIMIC DATA

In [3]:
mimic_path = '../data/MIMIC_data/'
input_path = '../models/doctorailib/'
output_path = '../data/doctorAI_input/'
pre_processing.prepare_mimic(mimic_path = mimic_path, input_path = input_path, output_path = output_path, CSS=True)

7e-05 % of diagnoses do not have the related ICD9 code associated (NaN values)
dropping the cases where there is no ICD9 code in the DIAGNOSES_ICD table

Created "clean_DIAGNOSES_ICD.csv" and "clean_ADMISSIONS.csv" in ../data/MIMIC_data/
Building pid-admission mapping, admission-date mapping
Building admission-dxList mapping
Building pid-sortedVisits mapping
Building pids, dates, strSeqs
Converting strSeqs to intSeqs, and making types
number of unique ICD9 codes in mimic: 4880
number of unique CSS-grouper codes in mimic: 272
Converting strSeqs to intSeqs, and making CCStypes

You can use the files visit and label (.train/.valid/.test) in ../data/doctorAI_input/ to train doctorAI:
seqFile="../data/doctorAI_input/visit"
labelFile="../data/doctorAI_input/label"
outFile="<output_path>/trained_drAI_model"
dr = doctorai.DoctorAI(ICD9_to_int_dict="../models/doctorailib//CCS_to_int_dict",verbose=True)
dr.train_doctorAI(seqFile=seqFile, inputDimSize=4880, labelFile=labelFile, numClass=272, outF

### Train doctorAI

In [4]:
dr = doctorai.DoctorAI(ICD9_to_int_dict="../models/ICD9_to_int_dict",CCS_to_int_dict="../models/CCS_to_int_dict",verbose=True)

In [5]:
seqFile="../data/doctorAI_input/visit"
labelFile="../data/doctorAI_input/label"
outFile="../models/trained_doctorAI_output/trained_drAI_model"

In [9]:
dr.train_doctorAI(seqFile=seqFile, 
                  inputDimSize=4880, 
                  labelFile=labelFile, 
                  numClass=272, 
                  outFile=outFile, 
                  max_epochs=50)

Initializing the parameters ... 
using randomly initialized code embedding
Building the model ... 
not using duration information, fine-tuning code representations
Loading data ... 
done
Optimization start !!
epoch:0, iteration:0/51, cost:68.453121
epoch:0, iteration:10/51, cost:62.047760
epoch:0, iteration:20/51, cost:55.284174
epoch:0, iteration:30/51, cost:60.895481
epoch:0, iteration:40/51, cost:52.706666
epoch:0, iteration:50/51, cost:55.653494
epoch:0, mean_cost:60.877149
Validation cross entropy:53.784212 at epoch:0
Test cross entropy:55.683485 at epoch:0
epoch:1, iteration:0/51, cost:62.629966
epoch:1, iteration:10/51, cost:50.970670
epoch:1, iteration:20/51, cost:49.778587
epoch:1, iteration:30/51, cost:53.765249
epoch:1, iteration:40/51, cost:61.089400
epoch:1, iteration:50/51, cost:51.991405
epoch:1, mean_cost:55.247387
Validation cross entropy:53.066720 at epoch:1
Test cross entropy:55.049088 at epoch:1
epoch:2, iteration:0/51, cost:48.467793
epoch:2, iteration:10/51, cost:

epoch:22, iteration:10/51, cost:53.958730
epoch:22, iteration:20/51, cost:50.667573
epoch:22, iteration:30/51, cost:55.324418
epoch:22, iteration:40/51, cost:56.475685
epoch:22, iteration:50/51, cost:54.309174
epoch:22, mean_cost:53.142601
Validation cross entropy:51.735853 at epoch:22
Test cross entropy:53.754042 at epoch:22
epoch:23, iteration:0/51, cost:50.089284
epoch:23, iteration:10/51, cost:60.078835
epoch:23, iteration:20/51, cost:57.614881
epoch:23, iteration:30/51, cost:54.343985
epoch:23, iteration:40/51, cost:51.924093
epoch:23, iteration:50/51, cost:56.338180
epoch:23, mean_cost:53.101723
Validation cross entropy:51.687433 at epoch:23
Test cross entropy:53.713142 at epoch:23
epoch:24, iteration:0/51, cost:50.551778
epoch:24, iteration:10/51, cost:48.805141
epoch:24, iteration:20/51, cost:63.799107
epoch:24, iteration:30/51, cost:53.398917
epoch:24, iteration:40/51, cost:55.706850
epoch:24, iteration:50/51, cost:48.756808
epoch:24, mean_cost:53.007263
Validation cross entro

epoch:45, iteration:10/51, cost:48.839273
epoch:45, iteration:20/51, cost:53.837080
epoch:45, iteration:30/51, cost:53.873728
epoch:45, iteration:40/51, cost:59.819632
epoch:45, iteration:50/51, cost:58.939204
epoch:45, mean_cost:51.498687
Validation cross entropy:51.005777 at epoch:45
Test cross entropy:52.999858 at epoch:45
epoch:46, iteration:0/51, cost:45.551125
epoch:46, iteration:10/51, cost:59.271425
epoch:46, iteration:20/51, cost:52.508800
epoch:46, iteration:30/51, cost:47.555107
epoch:46, iteration:40/51, cost:50.631264
epoch:46, iteration:50/51, cost:53.752281
epoch:46, mean_cost:51.451406
Validation cross entropy:51.025333 at epoch:46
epoch:47, iteration:0/51, cost:48.306769
epoch:47, iteration:10/51, cost:58.406007
epoch:47, iteration:20/51, cost:59.768610
epoch:47, iteration:30/51, cost:48.846316
epoch:47, iteration:40/51, cost:58.996413
epoch:47, iteration:50/51, cost:49.548805
epoch:47, mean_cost:51.430538
Validation cross entropy:51.058748 at epoch:47
epoch:48, iterat

### Test doctorAI performance

In [6]:
modelFile = outFile+'.45.npz'

dr.test_doctorAI(modelFile = modelFile, 
                 visit_test='../data/doctorAI_input/visit.test', 
                 label_test='../data/doctorAI_input/label.test', 
                 hiddenDimSize='[200,200]')

build model ... 
load data ... 
done
recall@10:0.35017931481317766, recall@20:0.5205799367686804, recall@30:0.6306969139940238


### Baselines performance

In [7]:
visit_file = pickle.load( open(output_path+'visit_complete', 'rb') )
label_file = pickle.load( open(output_path+'label_complete', 'rb') )

*From doctorAI paper*:

We compare our algorithms against simple baselines that are based on experts’ intuition about the dynamics of events in clinical settings. 

* The first baseline uses a patient’s medical codes in the last visit as the prediction for the current visit. This baseline is competitive when the status of a patient with a chronic condition stabilizes over time. 
* We enhanced this baseline using the top-k most frequent labels observed in visits prior to the current visits. In the experiments we observe that the baseline of top-k most frequent labels is quite competitive.

top-k recall = $\frac{\text{# of true positives in the top k predictions}}{\text{# of true positives}}$


Recall multilabel (https://stackoverflow.com/questions/9004172/precision-recall-for-multiclass-multilabel-classification):

The ratio of how many of the actual labels were predicted. The numerator finds how many labels in the predicted vector has common with the ground truth (as above), then finds the ratio to the number of actual labels, therefore getting what fraction of the actual labels were predicted.

In [8]:
def multi_label_recall(Y_true, Y_pred):
    #quanti sono gli items comuni tra predizione e truth?
    #quanti sono gli items che dovevo indovinare (lunghezza del vettore truth)?
    #divido il numero di items comuni per il numero di items che avrei dovuto azzeccare
    recall = []
    for y_true, y_pred in zip(Y_true,Y_pred):
        patient_recall = len(set(y_pred).intersection(set(y_true)))*1./len(set(y_true))
        recall.append(patient_recall)
    return np.mean(recall)

def recallTop(y_true, y_pred, rank=[10, 20, 30]):
    recall = list()
    for i in range(len(y_pred)):
        thisOne = list()
        codes = y_true[i]
        tops = y_pred[i]
        for rk in rank:
            thisOne.append(len(set(codes).intersection(set(tops[:rk])))*1.0/len(set(codes)))
        recall.append( thisOne )
    return (np.array(recall)).mean(axis=0).tolist()

#### Last visit is current visit

In [9]:
#last visit codes, this time codes
y_true = []
y_pred = []

for patient in label_file:
    n_visits = len(patient)
    current_visit = patient[n_visits-1]
    last_visit = patient[n_visits-2]
    y_true.append(current_visit)
    y_pred.append(last_visit)

In [10]:
multi_label_recall(y_true,y_pred)

0.44887523562649606

#### Most frequent

In [11]:
#top k most frequent labels in prior visits

In [12]:
y_pred = []
for patient in label_file:
    tops = sorted([x for x in set([a for b in patient[:-1] for a in b])],key=lambda y: ([a for b in patient[:-1] for a in b].count(y),y),reverse=True)
    y_pred.append(tops)

In [13]:
recallTop(y_true, y_pred)

[0.3828614784432503, 0.4732631519257384, 0.4909308495553468]